In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow tensorflowjs numpy pandas scikit-learn


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/input/phishing-url-dataset/phishing_dataset.csv")

# Example feature extraction
def make_features(url):
    length = min(len(url), 512)/512
    hasLogin = any(x in url.lower() for x in ["login","verify","secure"])
    dots = url.count(".") / 5
    at_sign = 1 if "@" in url else 0
    return [length, hasLogin, dots, at_sign, 0,0,0,0,0,0]  # pad to 10

X = np.array([make_features(u) for u in df['url']])
y = keras.utils.to_categorical(df['label'].astype(int), 3)

# Build model
model = keras.Sequential([
    keras.layers.Input(shape=(10,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X, y, validation_split=0.1, epochs=20, batch_size=64)

# Save model
model.save("/kaggle/working/phish_model.h5")


In [ ]:
!tensorflowjs_converter --input_format=keras /kaggle/working/phish_model.h5 /kaggle/working/phishing_model_tfjs
